In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score
# from matplotlib import pyplot as plt
# %matplotlib inline

# from utils import load_vectors, rnd_labeled_data_generator, test_data_generator, conseq_labeled_data_generator

Using TensorFlow backend.


Загружаем векторы

In [3]:
vectors = {
    "auto": "../data/auto_vectors.csv",
    "mus": "../data/mus_vectors.csv",
    "movies": "../data/movies_vectors.csv",
    "electr": "../data/electr_vectors.csv"
}

In [4]:
%%time
auto_df = load_vectors(vectors["auto"], manually=True)

'Reading line 20473'

Wall time: 10min 42s


In [8]:
auto_df.groupby("overall").count()

,vectors,target_bin
overall,,
1.0,542,542
2.0,606,606
3.0,1430,1430
4.0,3967,3967
5.0,13928,13928


Классификация с 5 категориями, не бинарная

In [8]:
auto_df["target"] = pd.get_dummies(auto_df["overall"]).values.tolist()
auto_df["target"] = auto_df["target"].apply(np.array)

In [9]:
auto_df.head()

,overall,vectors,target
0,5.0,"[[-0.67167366, 1.4340448, 0.70772099, 0.267104...","[0, 0, 0, 0, 1]"
1,4.0,"[[-3.4646065, 0.90924335, 0.051792312, -0.0250...","[0, 0, 0, 1, 0]"
2,5.0,"[[-0.34402627, -1.2019368, 0.49981338, -1.2384...","[0, 0, 0, 0, 1]"
3,5.0,"[[-1.187706, 0.5451839, 1.5461149, -0.60059845...","[0, 0, 0, 0, 1]"
4,5.0,"[[-0.78727818, 1.2093724, -2.3347111, -2.42030...","[0, 0, 0, 0, 1]"


Если делать zero-padding, очень быстро возникает MemoryError. Лучше учить по одному сэмплу

In [5]:
def max_len_padding(list_of_matrices):
    pairwise_len_max = lambda a, b: max(len(a), len(b))
    max_len = max([i.shape[0] for i in list_of_matrices])
    return max_len

def pad_by_zero_seqs_generator(list_of_matrices, y_list, timesteps, batch_size):
#     max_len = timesteps
#     aligned = [np.pad(i, ((0, max_len - i.shape[0]), (0, 0)), "constant", constant_values=0.0) for i in list_of_matrices]
#     aligned = np.array(aligned)
    while True:
        indices = np.random.choice(np.arange(len(list_of_matrices)), size=[batch_size])
        x = np.array(list_of_matrices)[indices]
        x = [np.pad(i, ((0, timesteps - i.shape[0]), (0, 0)), "constant", constant_values=0.0) for i in x]
        x = np.array(x)
        y = np.array(y_list)[indices].reshape([1, 1, 1])
        yield x, y

In [136]:
X_train, X_test, y_train, y_test = train_test_split(auto_df["vectors"].values, auto_df["target"].values, test_size=0.3)

In [137]:
y_test = np.dstack(y_test)[0].T
y_train = np.dstack(y_train)[0].T

In [33]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [300]:
# batch_size = 100
# timesteps = timesteps_padding(auto_df["vectors"].values)
# print("Timesteps:", timesteps)
# print("Batch size:", batch_size)
hidden_size1 = 80
hidden_size2 = 32

model = Sequential()
model.add(LSTM(hidden_size1, return_sequences=True, input_shape=(None, 128)))
# model.add(LSTM(hidden_size2, return_sequences=False))
model.add(Dense(5, activation="softmax"))
# model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='mean_absolute_error',
              optimizer='adagrad', metrics=["accuracy"])

model.fit_generator(rnd_labeled_data_generator(X_train, y_train), 
                    validation_data=rnd_labeled_data_generator(X_test, y_test), 
                    steps_per_epoch=30, epochs=150, verbose=1, validation_steps=100)

Epoch 1/150
30/30 [==============================] - 5s 161ms/step - loss: 0.3874 - acc: 0.8643 - val_loss: 0.3462 - val_acc: 0.8614
Epoch 2/150
30/30 [==============================] - 1s 38ms/step - loss: 0.3863 - acc: 0.8298 - val_loss: 0.3375 - val_acc: 0.8574
Epoch 3/150
30/30 [==============================] - 1s 38ms/step - loss: 0.3638 - acc: 0.8439 - val_loss: 0.3783 - val_acc: 0.8419
Epoch 4/150
30/30 [==============================] - 1s 37ms/step - loss: 0.2833 - acc: 0.8992 - val_loss: 0.3457 - val_acc: 0.8681
Epoch 5/150
30/30 [==============================] - 1s 34ms/step - loss: 0.3742 - acc: 0.8674 - val_loss: 0.2890 - val_acc: 0.8927
Epoch 6/150
30/30 [==============================] - 1s 30ms/step - loss: 0.3299 - acc: 0.8698 - val_loss: 0.3541 - val_acc: 0.8629
Epoch 7/150
30/30 [==============================] - 1s 35ms/step - loss: 0.3428 - acc: 0.8640 - val_loss: 0.3712 - val_acc: 0.8369
Epoch 8/150
30/30 [==============================] - 1s 40ms/step - loss: 0

30/30 [==============================] - 1s 37ms/step - loss: 0.2389 - acc: 0.9059 - val_loss: 0.3212 - val_acc: 0.8724
Epoch 124/150
30/30 [==============================] - 1s 30ms/step - loss: 0.2857 - acc: 0.8818 - val_loss: 0.3270 - val_acc: 0.8681
Epoch 125/150
30/30 [==============================] - 1s 33ms/step - loss: 0.3111 - acc: 0.8702 - val_loss: 0.2870 - val_acc: 0.8872
Epoch 126/150
30/30 [==============================] - 1s 29ms/step - loss: 0.3177 - acc: 0.8844 - val_loss: 0.3031 - val_acc: 0.8793
Epoch 127/150
30/30 [==============================] - 1s 33ms/step - loss: 0.3528 - acc: 0.8747 - val_loss: 0.2893 - val_acc: 0.8881
Epoch 128/150
30/30 [==============================] - 1s 37ms/step - loss: 0.2990 - acc: 0.8857 - val_loss: 0.2721 - val_acc: 0.8914
Epoch 129/150
30/30 [==============================] - 1s 38ms/step - loss: 0.3116 - acc: 0.8884 - val_loss: 0.3357 - val_acc: 0.8651
Epoch 130/150
30/30 [==============================] - 1s 37ms/step - loss: 

In [220]:
def predict(X, model):
    predictions = []
    for i, sample in enumerate(X):
        clear_output(True)
        display("Retrieving {}/{}".format(i + 1, X.shape[0]))
        predictions.append(model.predict(np.reshape(sample, [1, *sample.shape]), steps=1).reshape(-1, 5))
#     predictions = np.array(predictions)
#     predictions = np.reshape(predictions, (predictions.shape[0]))
    return predictions
    

model.predict() автоматически пытается склеить последовательности разной длины в один 2D-массив и ломается. Поэтому приходится ему тоже передавать входы по одному.

In [276]:
def result_to_one_hot(prediction):
    # Усреднение ответов для каждого слова внутри текста, потом выбор максимума
#     max_i = np.argmax(np.mean(prediction, axis=0).reshape(-1))
#     res = np.zeros(5)
#     res[max_i] = 1
#     return res.astype(int)
    # Выбор оценки исходя из самого значимого слова
#     max_i = np.argmax(np.max(prediction, axis=0))
#     res = np.zeros(5)
#     res[max_i] = 1
#     return res.astype(int)
    # Выбор по количеству максимальных
    # значений среди слов в пределах рубрики
    max_i = np.argmax(np.argmax(prediction, axis=0))
    res = np.zeros(5)
    res[max_i] = 1
    return res.astype(int)


def decode_one_hot(oh_matrix):
    return np.where(oh_matrix == 1)[1] + 1

In [302]:
res = predict(X_test, model)

'Retrieving 6142/6142'

In [303]:
oh_res = np.array(list(map(result_to_one_hot, res)))

In [304]:
print("Balanced accuracy:\t\t{}".format(balanced_accuracy_score(decode_one_hot(y_test), 
                                                                         decode_one_hot(oh_res), adjusted=False)))
print("Balanced and adjusted accuracy:\t{}".format(balanced_accuracy_score(decode_one_hot(y_test), 
                                                                         decode_one_hot(oh_res), adjusted=True)))
print("Unbalanced accuracy:\t\t{}".format(accuracy_score(decode_one_hot(y_test), 
                                                         decode_one_hot(oh_res))))

Balanced accuracy:		0.2901629895562134
Balanced and adjusted accuracy:	0.11270373694526672
Unbalanced accuracy:		0.4317811787691306


In [298]:
decode_one_hot(y_test[:10])

array([5, 5, 5, 5, 5, 5, 4, 4, 4, 5], dtype=int64)

In [299]:
decode_one_hot(oh_res[:10])

array([5, 5, 4, 2, 3, 5, 5, 1, 5, 5], dtype=int64)

In [294]:
oh_res[:10]

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

При классификации по 5 рубрикам качество остается низким при различных конфигурациях модели.
Попробуем перейти к бинарной классификации.

In [9]:
auto_df["target_bin"] = (auto_df["overall"] > 3).astype(int)
auto_df.head()

,overall,vectors,target_bin
0,5.0,"[[-0.67167366, 1.4340448, 0.70772099, 0.267104...",1
1,4.0,"[[-3.4646065, 0.90924335, 0.051792312, -0.0250...",1
2,5.0,"[[-0.34402627, -1.2019368, 0.49981338, -1.2384...",1
3,5.0,"[[-1.187706, 0.5451839, 1.5461149, -0.60059845...",1
4,5.0,"[[-0.78727818, 1.2093724, -2.3347111, -2.42030...",1


Классы очень несбалансированны:

In [10]:
auto_df.groupby("target_bin").count()

,overall,vectors
target_bin,,
0,2578,2578
1,17895,17895


In [11]:
ind_to_drop = np.random.choice(auto_df[auto_df["target_bin"] == 1].index, 
                               size=(auto_df.shape[0] - 2*auto_df[auto_df["target_bin"] == 0].shape[0]), 
                               replace=False)

In [12]:
all(auto_df.loc[ind_to_drop, "target_bin"])

True

In [13]:
ind_to_drop.shape, auto_df.shape

((15317,), (20473, 3))

In [14]:
np.unique(auto_df.index).shape 

(20473,)

In [15]:
balanced_df = auto_df.drop(ind_to_drop, axis=0)
balanced_df.groupby("target_bin").count()

,overall,vectors
target_bin,,
0,2578,2578
1,2578,2578


In [16]:
X_train, X_test, y_train, y_test = train_test_split(balanced_df["vectors"].values, 
                                                    balanced_df["target_bin"].values, test_size=0.3)

In [17]:
def predict_bin(X, model):
    predictions = []
    for i, sample in enumerate(X):
        clear_output(True)
        display("Retrieving {}/{}".format(i + 1, X.shape[0]))
        predictions.append(model.predict(np.reshape(sample, [1, *sample.shape]), steps=1).reshape(-1))
    return predictions

def res_to_bin(y):
    return np.array(list(map(lambda a: np.round(np.mean(a)).astype(int), y)))

In [18]:
hidden_size1 = 32
hidden_size2 = 150

model = Sequential()
model.add(LSTM(hidden_size1, return_sequences=True, input_shape=(None, 128)))
# model.add(LSTM(hidden_size2, return_sequences=True))
# model.add(Dense(hidden_size2, activation="hard_sigmoid"))
model.add(Dense(1, activation='hard_sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adagrad', metrics=["accuracy"])

model.fit_generator(rnd_labeled_data_generator(X_train, y_train), 
                    validation_data=rnd_labeled_data_generator(X_test, y_test), 
                    steps_per_epoch=1500, epochs=2, verbose=1, validation_steps=100)

Epoch 1/2
1500/1500 [==============================] - 33s 22ms/step - loss: 0.6237 - acc: 0.6582 - val_loss: 0.5873 - val_acc: 0.6843
Epoch 2/2
1500/1500 [==============================] - 33s 22ms/step - loss: 0.5760 - acc: 0.7072 - val_loss: 0.5295 - val_acc: 0.7289


In [21]:
res = predict_bin(X_test, model)
bin_res = res_to_bin(res)

'Retrieving 1547/1547'

In [22]:
np.bincount(bin_res)

array([744, 803], dtype=int64)

In [23]:
print("Balanced accuracy:\t\t{}".format(balanced_accuracy_score(y_test, bin_res, adjusted=False)))
print("Balanced and adjusted accuracy:\t{}".format(balanced_accuracy_score(y_test, bin_res, adjusted=True)))
print("Unbalanced accuracy:\t\t{}".format(accuracy_score(y_test, bin_res,)))

Balanced accuracy:		0.7475395380589374
Balanced and adjusted accuracy:	0.4950790761178747
Unbalanced accuracy:		0.7478991596638656


Кернель умирает при сохранении модели. Файл экспортирован в .py

In [ ]:
model.save("classifier.hdf5")